Plot the locations of the profiles for the different regions. 

In [55]:
import os
import glob
import netCDF4
import logging
import numpy as np
import emodnetchemistry
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import calendar
import warnings
import matplotlib.cbook
from matplotlib.ticker import FormatStrFormatter
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams.update({'font.size': 16})
plt.rc('figure', facecolor='w')
from importlib import reload
reload(emodnetchemistry)

<module 'emodnetchemistry' from '/home/ctroupin/Projects/EMODnet/Chemistry3/EMODnet-Chemistry/python/emodnetchemistry.py'>

In [2]:
logger = logging.getLogger("EMODnet-Chemistry-Data")
logger.setLevel(logging.DEBUG)
logging.info("Starting")

## Paths

In [6]:
datadir = "/data/EMODnet/Eutrophication/Split/"
figdir = "/home/ctroupin/dox/EMODnet-Chemistry/Histograms/"
if not os.path.exists(figdir):
    os.makedirs(figdir)
varlist = ["phosphate", "silicate", "ammonium", "chlorophyll-a", 
           "dissolved_inorganic_nitrogen", "dissolved_oxygen"]

## Loop on the variables

In [12]:
for varname in varlist[3:4]:
    logger.info("Working on variable {}".format(varname))
    datafilelist = sorted(glob.glob(os.path.join(datadir, f"*{varname}*.nc")))
    nfiles = len(datafilelist)
    logger.info("Working on {} files".format(nfiles))
    datafilelist

    timeobs = np.array([])
    dateobs = np.array([])
    obsdepth = np.array([])
    obsval = np.array([])
    for datafile in datafilelist:
        logger.info("Working on {}".format(datafile))
        with netCDF4.Dataset(datafile, "r") as nc:
            
            varnamenc = (list(nc.variables.keys())[-1])
            logger.info("Name in netCDF file: {}".format(varnamenc))
            timeunits = nc.variables["obstime"].units
            times = nc.variables["obstime"][:]
            depth = nc.variables["obsdepth"][:]
            values = nc.variables[varnamenc][:]
            dates = netCDF4.num2date(times, timeunits)
            timeobs = np.append(timeobs, times)
            dateobs = np.append(dateobs, dates)
            obsval = np.append(obsval, values)
            obsdepth = np.append(obsdepth, depth)
        
    logger.info("Preparing arrays of years and months")
    years = [dd.year for dd in dateobs.data]
    months = [dd.month for dd in dateobs.data]
    
    logger.info("Creating histograms")
    #emodnetchemistry.make_histo_values(obsval, varname, figdir)
    #emodnetchemistry.make_histo_month(months, varname, figdir)
    #emodnetchemistry.make_histo_year(years, varname, figdir)

INFO:EMODnet-Chemistry-Data:Working on variable chlorophyll-a
INFO:EMODnet-Chemistry-Data:Working on 8 files
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/ArcticSea_Water_body_chlorophyll-a.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body chlorophyll-a
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/Atlantic_TS_Water_body_chlorophyll-a.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body chlorophyll-a
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/Atlantic_Water_body_chlorophyll-a.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body chlorophyll-a
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/BalticSea_Water_body_chlorophyll-a.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body chlorophyll-a
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/BlackSea_Water_body_chlorophyll-a.nc
INFO:EMODnet-Chemistry-Data:Name in

## Depth polar plot

In [23]:
months = np.array(months)

In [70]:
def plot_month_depth_ndata(obsdepth, months, figname=None, depthlist = [5., 100., 500., 1000., 2000.]):
    
    angles = np.arange(0, 2. * np.pi + .1, np.pi / 6.)
    
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.grid(color='.85', linestyle = '--', linewidth=1)
    ax.set_thetagrids(np.arange(0, 360, 30), 
                      labels=[calendar.month_name[ii] for ii in range(1, 13)], 
                      fontsize=20)
    ax.set_rlabel_position(2)
    ax.set_rgrids(np.arange(250000, 1500000, 250000))

    for dd in depthlist:
        depthselector = np.where(obsdepth <= dd)[0]
        monthsdepth = months[depthselector]

        ndatamonth = np.zeros(13)
        for mm in range(0, 12):
            # logger.info(mm)
            monthselector = np.where(monthsdepth == mm+1)[0]
            ndatamonth[mm] = len(monthselector)
        ndatamonth[-1] = ndatamonth[0]

        ax.plot(angles, ndatamonth, "o-", ms=5, label="< {} m".format(int(dd)))

    ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.legend(bbox_to_anchor=(1.2, 1.1))
    plt.title("Number of measurements of {}".format(varname), fontsize=24)
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight", facecolor="w", transparent=False)
    plt.close()

In [71]:
plot_month_depth_ndata(obsdepth, months, os.path.join(figdir, "month_depth_histo_{}".format(varname)))